In [ ]:
import numpy as np
import pandas as pd
import sys

libpath = './../../bgl_model/'
sys.path.append(libpath)

from main_model import *

In [ ]:
sigma_noise = 5  # noise std. dev.
ins_ceiling = 45  # max. insulin dose 
ins_spacing = 0.1
bg_omin = 70  # hypo boundary
bg_omax = 180  # hyper boundary
postbg_target = 112.5  # target dose

global_tmi = 0.0  # we assume the patient takes the bolus dose with the meal
target_tmbg = 150  # makes recommendations to optimize the PPBG after 150 mins from the meal

num_all_train = 20  # num. of available training events per patient
num_all_test = 20  # num. of available test events per patient

all_patients = ['adult#002', 'adult#005', 'adult#007', 'adult#008', 'adult#010']

clinician_recs = {'adult#002': [6.0, 4.5, 7.2, 2.7, 4.1, 5,6.5, 5.6, 5.4, 4.5, 12.2, 6.1, 11.9, 6.9, 2.7, 5.6, 6.8, 8.8, 6.8, 5.9],
                  'adult#005': [2.9, 11.8, 17.3, 19.9, 15.8, 15.2, 8.8,15.4, 7.8, 3, 12.7, 12.2, 4.5, 11.9, 7.8, 10.9, 8.8,26.7, 9.8, 8.5],
                  'adult#007': [1.4, 2.8, 3.6, 2.8, 3.2, 1.8, 0.9, 3.8, 3.2, 3.6, 6.3, 2.6, 2, 2.2, 1.7, 2.9, 1, 3.5, 3.4, 2.5],
                  'adult#008': [2.7, 3.4, 2.4, 1.4, 4.3, 2.8, 2.2, 2.8, 2.2, 0.9, 1.1, 1.7, 3, 3, 0.7, 2.5, 4.4, 2.1, 5.5, 1.3,],
                  'adult#010': [5.4, 17.8, 12.3, 13.6, 16.1, 12.2, 8.2, 10, 22.9, 24.8, 6.7, 33.1, 41.5, 6.7, 4.8, 14.5, 11.2, 15.9, 18.9, 9]}

In [ ]:
def std_bolus_calc(cho, gm, gt, cr, cf, iob=None):
    _iob = iob if iob else 0
    return np.max([0, cho / cr + (gm - gt) / cf - _iob])

In [ ]:
def create_model(data_init, ins_ceiling, postbg_target, bg_omin, bg_omax):
    variables = [{'name': 'meal', 'domain': (20, 80), 'linearbounds':(0,9), 'unit': 'g'},
                 {'name': 'tmbg', 'domain': (120, 210), 'linearbounds':(0,9), 'unit': 'min'},
                 {'name': 'bg_fasting', 'domain': (100, 150), 'linearbounds':(0,9), 'unit': 'mg/dl'},
                 {'name': 'insulin', 'domain': (0, ins_ceiling), 'linearbounds':(-9,0), 'unit': 'U'}]

    bgl_model = BGL_MODEL(postBG_target_range=(bg_omin, bg_omax), postBG_target_value=postbg_target, variables=variables)
    
    for ind in range(len(data_init)):
        new_x = np.array([[data_init.loc[ind,'meal']], [data_init.loc[ind,'tmbg']], [data_init.loc[ind,'bg_fasting']], [data_init.loc[ind,'insulin']]]).reshape(1,-1)
        new_m = np.array([data_init.loc[ind,'bg_postprandial']]).reshape(-1,1)
        bgl_model.update_lts(new_x, new_m)

    return bgl_model

In [ ]:
num_train = 20  # num. of training events per patient to be used
num_test = 20 # num. of test events per patient

test_dict = {}  # dict to store test results for each patient

for cur_patient in all_patients:

    bgl_vals = np.load('./../calc_res_clinician_data/{}_calc_res.npy'.format(cur_patient))

    df = pd.read_csv('./../Quest.csv')
    cr = df.loc[df['Name']==cur_patient]['CR'].values[0] 
    cf = df.loc[df['Name']==cur_patient]['CF'].values[0]


    train_data = pd.read_csv('./train_data/train_{}.csv'.format(cur_patient))
    test_data = pd.read_csv('./test_data/test_{}.csv'.format(cur_patient))

    bgl_model_salelts = create_model(train_data, ins_ceiling=ins_ceiling, postbg_target=postbg_target, bg_omin=70, bg_omax=180)

    for i in range(num_test):
        print('\nPatient: {}, Meal Event: {}'.format(cur_patient, i + 1))

        context={'meal': test_data['meal'][i],
                 'tmbg': target_tmbg, 
                 'bg_fasting': test_data['bg_fasting'][i]}
            
        ins_calc = min(ins_ceiling, std_bolus_calc(cho=context['meal'], gm=context['bg_fasting'], gt=112.5, cr=cr, cf=cf))
        ins_salelts = min(ins_ceiling, bgl_model_salelts.recommend(context, strategy='salelts', ins_calc=ins_calc, ins_ceiling=ins_ceiling, aux_ins=10, S=20, L=200, lam=0.1, exp_coeff=0.01))
        ins_cli = min(ins_ceiling, clinician_recs[cur_patient][i])
     
        bgl_calculator = bgl_vals[i, int(ins_calc/ins_spacing)] + np.random.normal(loc=0, scale=sigma_noise)
        bgl_salelts = bgl_vals[i, int(ins_salelts/ins_spacing)] + np.random.normal(loc=0, scale=sigma_noise)
        bgl_cli = bgl_vals[i, int(ins_cli/ins_spacing)] + np.random.normal(loc=0, scale=sigma_noise)

        test_data.loc[i,'insulin_calc'] = ins_calc
        test_data.loc[i,'insulin_salelts'] = ins_salelts
        test_data.loc[i,'insulin_cli'] = ins_cli
        test_data.loc[i,'SALE-LTS'] = bgl_salelts
        test_data.loc[i,'Calculator'] = bgl_calculator
        test_data.loc[i,'Clinicians'] = bgl_cli

    test_data.to_csv('./test_res/test_res_{}.csv'.format(cur_patient), index=False)
    test_dict['{}'.format(cur_patient)] = test_data

all_test = pd.concat(list(test_dict.values()), ignore_index=True)
all_test.to_csv('./test_res/test_res.csv', index=False)